### Install and import libraries

In [3]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 KB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 23.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import libraries
import torch
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM

# check transformers version
MIN_TRANSFORMERS_VERSION = '4.25.1'
assert transformers.__version__ >= MIN_TRANSFORMERS_VERSION, f'Please upgrade transformers to version {MIN_TRANSFORMERS_VERSION} or higher.'

torch.set_printoptions(precision=2, threshold=3000)
transformers.utils.logging.set_verbosity_error()

In [5]:
import bitsandbytes

### Download and load an open source model

https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

https://www.together.xyz/blog/redpajama-7b

https://huggingface.co/togethercomputer/RedPajama-INCITE-Instruct-3B-v1

In [6]:
# load model
config = AutoConfig.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1", device_map='auto', torch_dtype=torch.float16, load_in_8bit=False)
model = model.to('cuda:0')

In [7]:
# infer
from IPython.display import Markdown

def generate_answer(question, max_new_tokens=256):
    prompt = f"<human>: {question}\n<bot>:"
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_length = inputs.input_ids.shape[1]
    outputs = model.generate( **inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True)
    tokens = outputs.sequences[0, input_length:]
    answer = tokenizer.decode(tokens)
    return Markdown("### Question\n" + question + "\n### Réponse\n" + answer)

### Test the local language model

In [11]:
generate_answer("Comment financer un achat de maison ?")

### Question
Comment financer un achat de maison ?
### Réponse
 Pour financer un achat de maison, il faut savoir :

- La valeur de la maison : La valeur de la maison dépendra des options et des options de l'achat.

- La situation financière : Les revenus et les créances de l'utilisateur doivent être connus pour financer l'achat.

- La durée de la dette : La durée de la dette doit être prise en compte pour calculer le taux d'intérêt.

- La période de la dette : La période de la dette doit être prise en compte pour calculer le taux d'intérêt.

- Les options : Les options peuvent être utilisées pour financer l'achat de la maison, mais il faut savoir les options et les options de l'achat.

- Les taxes et impôts : Les taxes et impôts peuvent être facteurs à prendre en compte pour financer l'achat de la maison.

- Les coûts : Les coûts de construction et d'achat peuvent

In [24]:
generate_answer("En python, comment trier les éléments d'un dictionnaire selon leur valeur décroissante ?")

### Question
En python, comment trier les éléments d'un dictionnaire selon leur valeur décroissante ?
### Réponse
 Pour trier les éléments d'un dictionnaire en fonction de leur valeur décroissante, vous pouvez utiliser le module `sorted` du module `collections`.

Voici un exemple de code qui utilise `sorted` pour trier les éléments d'un dictionnaire en fonction de leur valeur décroissante :

```python
def sort_dict(dictionary):
    return sorted(dictionary.sort(key=lambda x: x[1]))
```

Vous pouvez utiliser ce module pour trier les éléments d'un dictionnaire en fonction de leur valeur décroissante en utilisant la fonction `sort` et en spécifiant les valeurs à trier.

Voici un exemple de code qui utilise `sort_dict` pour trier les éléments d'un dictionnaire en fonction de leur valeur décroissante :

```python
import sorted

# Définir le dictionnaire
dictionary = {"a": [1, 2, 3], "b": [4, 5, 6], "c": [7, 8, 9]}


### Source code of the model : 

https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neox/modeling_gpt_neox.py

In [12]:
question = "Comment financer un achat de maison ?"

#### 1. Tokenizer - Vocabulary

In [13]:
len(tokenizer.vocab)

50277

In [14]:
i = 0
for item in sorted(tokenizer.vocab.items(), key=lambda x: x[1]):
    i += 1
    if i<=10000:
        continue
    print(f"{item[1]} -> '{item[0].replace('Ġ',' ')}'")
    if i>10020:
        break

10000 -> ' busy'
10001 -> '$_'
10002 -> 'who'
10003 -> ' institutions'
10004 -> 'Select'
10005 -> ' progression'
10006 -> ' modes'
10007 -> ' modify'
10008 -> 'description'
10009 -> ' disag'
10010 -> 'ctic'
10011 -> ' SC'
10012 -> ' theorem'
10013 -> ' grab'
10014 -> ' theme'
10015 -> ' clothes'
10016 -> '2002'
10017 -> ' replied'
10018 -> ' observe'
10019 -> 'iginal'
10020 -> ' semi'


#### 2. Tokenizer - Tokens and token ids

In [15]:
 tokenizer

GPTNeoXTokenizerFast(name_or_path='togethercomputer/RedPajama-INCITE-Chat-3B-v1', vocab_size=50254, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [16]:
def get_tokens(question):
    tokens_ids = tokenizer(question)["input_ids"]
    return "[" + "] [".join([tokenizer.decode(token_id) for token_id in tokens_ids]) + "]"
    
def get_token_ids(question):
    tokens_ids = tokenizer(question)["input_ids"]
    return "[" + "] [".join([str(token_id) for token_id in tokens_ids]) + "]"

In [17]:
get_tokens(question)

'[Comment] [ fin] [ancer] [ un] [ a] [chat] [ de] [ ma] [ison] [?]'

In [18]:
get_token_ids(question)

'[19174] [1442] [21955] [440] [247] [23481] [372] [6429] [1988] [3736]'

#### 3. Embeddings - Token representations

In [19]:
model.gpt_neox.embed_in

Embedding(50432, 2560)

In [22]:
def get_embeddings(question):
    input_ids = torch.tensor(tokenizer(question)["input_ids"]).cuda()
    return model.gpt_neox.embed_in(input_ids)

In [23]:
input_embeddings = get_embeddings(question)

In [24]:
input_embeddings.size()

torch.Size([10, 2560])

In [25]:
input_embeddings[0,:200]

tensor([ 0.01, -0.01, -0.02, -0.02,  0.04, -0.01, -0.03, -0.01, -0.03,  0.01,
        -0.01, -0.00,  0.02, -0.01,  0.04,  0.02,  0.05,  0.04, -0.01,  0.01,
         0.01, -0.01, -0.02, -0.03,  0.00,  0.04,  0.00,  0.02, -0.04,  0.00,
        -0.07, -0.01, -0.01,  0.03, -0.02, -0.03,  0.00,  0.01, -0.01, -0.02,
         0.02,  0.01, -0.02,  0.03,  0.00,  0.02,  0.01,  0.01, -0.01, -0.01,
         0.01, -0.01,  0.01,  0.03,  0.02,  0.03,  0.01, -0.03, -0.01,  0.00,
         0.03,  0.04,  0.03,  0.02, -0.01, -0.01,  0.01, -0.02, -0.00, -0.01,
         0.00,  0.01,  0.02,  0.03,  0.01, -0.02,  0.00, -0.04,  0.01, -0.02,
         0.02,  0.04, -0.03,  0.01,  0.01,  0.03, -0.00, -0.02, -0.00,  0.01,
        -0.02,  0.01,  0.01,  0.02,  0.03,  0.02, -0.02,  0.02,  0.03,  0.01,
        -0.01,  0.00, -0.00,  0.02,  0.01, -0.00,  0.03, -0.00,  0.01,  0.03,
        -0.00,  0.01, -0.01, -0.01, -0.02, -0.00, -0.01,  0.02, -0.02, -0.02,
        -0.02, -0.02,  0.01, -0.01,  0.02, -0.01, -0.02,  0.00, 

#### 4. Transformer Layers - Contextual & semantic representations

In [26]:
model.gpt_neox.layers[0]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  (attention): GPTNeoXAttention(
    (rotary_emb): RotaryEmbedding()
    (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
    (dense): Linear(in_features=2560, out_features=2560, bias=True)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
    (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
    (act): GELUActivation()
  )
)

In [27]:
model.gpt_neox.layers[31]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  (attention): GPTNeoXAttention(
    (rotary_emb): RotaryEmbedding()
    (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
    (dense): Linear(in_features=2560, out_features=2560, bias=True)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
    (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
    (act): GELUActivation()
  )
)

#### 5. Language modeling head - Predicting the next token id

In [29]:
model.embed_out

Linear(in_features=2560, out_features=50432, bias=False)

In [32]:
def get_predictions(question):
    input_ids = torch.tensor(tokenizer(question)["input_ids"]).unsqueeze(dim=0).cuda()
    return model(input_ids).logits.squeeze().float()

In [33]:
preds = torch.softmax(get_predictions(question), dim=1)

In [34]:
preds.size()

torch.Size([10, 50432])

In [35]:
next_token_prediction = preds[-1,:]
next_token_prediction

tensor([3.55e-02, 1.83e-10, 2.20e-03,  ..., 1.74e-10, 1.39e-10, 1.42e-10],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [36]:
next_token_ids = torch.topk(next_token_prediction, k=10)
next_token_ids

torch.return_types.topk(
values=tensor([0.75, 0.04, 0.03, 0.01, 0.01, 0.00, 0.00, 0.00, 0.00, 0.00],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([  187,     0, 22955,   330,  2070,   313,  3277, 12029,   428,  3905],
       device='cuda:0'))

In [37]:
[tokenizer.decode(token_id) for token_id in next_token_ids.indices]

['\n',
 '<|endoftext|>',
 ' Comment',
 ' C',
 ' Le',
 ' (',
 ' Qu',
 ' Les',
 ' -',
 ' La']

#### 6. Observe the inner workings of the model - Iterate to sample an answer token by token

In [40]:
if 'hook_handles' not in vars():
    hook_handles = []
else:
    for hook_handle in hook_handles:
        hook_handle.remove()
        hook_handles = []

def get_input_tokens(inpTensor):
    return "[" + "] [".join([tokenizer.decode(element.item()).replace('\n',"\\n") for element in inpTensor]) + "]"

def get_output_tokens(outTensor):
    preds = torch.softmax(outTensor.float(), dim=1)
    next_token_ids = torch.topk(preds[-1,:], k=5)
    return [tokenizer.decode(token_id) for token_id in next_token_ids.indices]

In [41]:
for hook_handle in hook_handles:
    hook_handle.remove()
hook_handles = []

def print_embed_in(module, input, output):
    inpTensor = input[0].squeeze(dim=0)
    print(f">> Input : {get_input_tokens(inpTensor)}")
    
def print_embed_out(module, input, output):
    outTensor = output[0]
    print(f">> Output:  {get_output_tokens(outTensor)}")

# Register the hook to a specific layer/module
hook_handles.append(model.gpt_neox.embed_in.register_forward_hook(print_embed_in))
hook_handles.append(model.embed_out.register_forward_hook(print_embed_out))

In [42]:
generate_answer(question)

>> Input : [<] [human] [>:] [ Comment] [ fin] [ancer] [ un] [ a] [chat] [ de] [ ma] [ison] [?] [\n] [<] [bot] [>:]
>> Output:  [' Il', ' Pour', ' Vo', ' F', ' V']
>> Input : [ Il]
>> Output:  [' existe', ' est', ' faut', ' y', ' n']
>> Input : [ est]
>> Output:  [' important', ' possible', ' diff', ' très', ' sou']
>> Input : [ important]
>> Output:  [' de', ' d', ' que', ' pour', ',']
>> Input : [ de]
>> Output:  [' cho', ' comp', ' sav', ' p', ' se']
>> Input : [ comp]
>> Output:  ['rendre', 'ter', 'rim', 'ren', 'render']
>> Input : [rendre]
>> Output:  [' les', ' la', ' le', ' comment', ' ce']
>> Input : [ les]
>> Output:  [' co', ' diffé', ' dé', ' é', ' avant']
>> Input : [ diffé]
>> Output:  ['rent', 'rences', 'rends', 'ren', 'rens']
>> Input : [rent]
>> Output:  ['s', 'es', ' types', ' options', ' su']
>> Input : [s]
>> Output:  [' fact', ' él', ' aspects', ' types', ' co']
>> Input : [ fact]
>> Output:  ['eurs', 'eur', 'ures', 'urs', 'ure']
>> Input : [eurs]
>> Output:  [' qui'

### Question
Comment financer un achat de maison ?
### Réponse
 Il est important de comprendre les différents facteurs qui peuvent affecter la viabilité d'un achat de maison.

1. - Valeur du terrain : La valeur du terrain est importante car elle peut être la principale source de revenus pour votre maison. Si le terrain est bien situé, il peut être plus facile à gérer et à améliorer.

2. - Prix de la location : La location d'un terrain peut être un facteur important pour les investisseurs. Il peut être plus facile de louer une maison si le terrain est bien situé.

3. - Créer un patrimoine : Le patrimoine est important pour les investisseurs car il peut permettre de gérer les coûts de construction et d'entretien plus aisément.

4. - Durée de la construction : La durée de la construction peut être un facteur important pour les investisseurs car elle peut affecter la rentabilité de la construction.

5. - Taxes : Les taxes sur la location d'un terrain peuvent être important pour les investisseurs